# Experimenting with Feedback Analysis using Sentiment Analysis and Collaborative Filtering

## Data

### Child Feature Data

The dataset comprises generated data on 30 children, including features such as age, gender, and health considerations. These features are known to influence food preferences significantly. Such data could be collected through an app or existing school databases. Over time, with enough data, these features could be used to predict whether a new individual will like a particular food.

In [6]:
def get_child_database():
    child_features = {
        "child1": {"age": 10, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child2": {"age": 8, "gender": "F", "health_consideration": "very health conscious", "favorite_cuisine": "Italian"},
        "child3": {"age": 9, "gender": "M", "health_consideration": "moderately health conscious", "favorite_cuisine": "Italian"},
        "child4": {"age": 7, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child5": {"age": 11, "gender": "M", "health_consideration": "moderately health conscious", "favorite_cuisine": "Italian"},
        "child6": {"age": 12, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child7": {"age": 9, "gender": "M", "health_consideration": "moderately health conscious", "favorite_cuisine": "Italian"},
        "child8": {"age": 8, "gender": "F", "health_consideration": "very health conscious", "favorite_cuisine": "Italian"},
        "child9": {"age": 10, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child10": {"age": 11, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child11": {"age": 7, "gender": "F", "health_consideration": "moderately health conscious", "favorite_cuisine": "Italian"},
        "child12": {"age": 9, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child13": {"age": 8, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Chinese"},
        "child14": {"age": 10, "gender": "M", "health_consideration": "moderately health conscious", "favorite_cuisine": "Chinese"},
        "child15": {"age": 12, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Chinese"},
        "child16": {"age": 11, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "Chinese"},
        "child17": {"age": 9, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Italian"},
        "child18": {"age": 7, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "Chinese"},
        "child19": {"age": 8, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Chinese"},
        "child20": {"age": 10, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "British"},
        "child21": {"age": 10, "gender": "F", "health_consideration": "very health conscious", "favorite_cuisine": "British"},
        "child22": {"age": 7, "gender": "M", "health_consideration": "moderately health conscious", "favorite_cuisine": "British"},
        "child23": {"age": 9, "gender": "F", "health_consideration": "moderately health conscious", "favorite_cuisine": "British"},
        "child24": {"age": 8, "gender": "M", "health_consideration": "don't care", "favorite_cuisine": "British"},
        "child25": {"age": 11, "gender": "F", "health_consideration": "very health conscious", "favorite_cuisine": "British"},
        "child26": {"age": 12, "gender": "M", "health_consideration": "moderately health conscious", "favorite_cuisine": "British"},
        "child27": {"age": 9, "gender": "F", "health_consideration": "moderately health conscious", "favorite_cuisine": "Chinese"},
        "child28": {"age": 7, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Chinese"},
        "child29": {"age": 8, "gender": "M", "health_consideration": "very health conscious", "favorite_cuisine": "Italian"},
        "child30": {"age": 10, "gender": "F", "health_consideration": "don't care", "favorite_cuisine": "Italian"}
    }
    return child_features


### Child Preference Data
The dataset comprises generated preferences for 30 children. In a real-world scenario, such data could be collected through questionnaires or feedback after meals. This serves as the starting point for this notebook. For this example, the data is generated based on a study that discovered vegetables are grouped according to dislike. Each child is assigned a chance of liking, being neutral, or disliking all ingredients within a group. These probabilities are roughly estimated from the research paper.

Reference: [Study on children's vegetable preferences](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6173934/)


In [39]:
import random

def initialize_children_data(child_features):
    # Define vegetable groups with descriptions
    vegetable_groups = {
        "Group A": ["tomato", "corn", "pumpkin", "carrot"],
        "Group B": ["onion", "red pepper"],
        "Group C": ["cauliflower"],
        "Group D": ["zucchini", "spinach", "chard", "peas"],
        "Group E": ["beetroot", "lettuce", "broccoli"],
        "Group F": ["eggplant", "cucumber", "cabbage", "green beans"]
    }

    # Define probabilities based on group descriptions
    probabilities = {
        "Group A": {"like": 0.7, "neutral": 0.2, "dislike": 0.1},  # Most liked
        "Group B": {"like": 0.4, "neutral": 0.3, "dislike": 0.3},  # Camouflaged in recipes
        "Group C": {"like": 0.1, "neutral": 0.2, "dislike": 0.7},  # Strongly disliked
        "Group D": {"like": 0.3, "neutral": 0.3, "dislike": 0.4},  # Camouflaged but some forced
        "Group E": {"like": 0.2, "neutral": 0.3, "dislike": 0.5},  # Offered but often rejected
        "Group F": {"like": 0.2, "neutral": 0.3, "dislike": 0.5}   # Rarely offered, often rejected
    }

    children_data = {}

# Iterate over each child in the dataset
    for child_key, features in child_features.items():
        # Get the health consideration feature for the current child
        health_consideration = features["health_consideration"]
        
        # Calculate like probability modifier based on health consideration
        # More health conscious children have a higher modifier, making them less likely to dislike vegetables
        if health_consideration == "very health conscious":
            like_modifier = 0.2
        elif health_consideration == "moderately health conscious":
            like_modifier = 0.1
        else:
            like_modifier = 0

        # Initialize the preferences for the current child
        preferences = {"likes": [], "neutral": [], "dislikes": []}

        # Assign preferences based on group probabilities and health consideration
        for group, veggies in vegetable_groups.items():
            for veggie in veggies:
                # Generate a random value between 0 and 1
                rand_val = random.random()
                # Get the probability distribution for the current group
                prob = probabilities[group]
                
                # Determine preference based on probabilities and health consideration modifier
                if rand_val < prob["like"] - like_modifier:
                    preferences["likes"].append(veggie)
                elif rand_val < prob["like"] + prob["neutral"]:
                    preferences["neutral"].append(veggie)
                else:
                    preferences["dislikes"].append(veggie)

        # Store the preferences for the current child
        children_data[child_key] = preferences
    
    return children_data

child_features = get_child_database()
children_data = initialize_children_data(child_features)
# print(children_data)
for child, preferences in children_data.items():
    print(f"{child}: {preferences}")



child1: {'likes': ['corn', 'pumpkin', 'onion', 'cauliflower', 'chard', 'cabbage'], 'neutral': ['tomato', 'carrot', 'zucchini', 'peas', 'broccoli'], 'dislikes': ['red pepper', 'spinach', 'beetroot', 'lettuce', 'eggplant', 'cucumber', 'green beans']}
child2: {'likes': ['corn', 'pumpkin', 'carrot', 'onion'], 'neutral': ['tomato', 'red pepper', 'chard', 'peas', 'lettuce', 'eggplant', 'cabbage', 'green beans'], 'dislikes': ['cauliflower', 'zucchini', 'spinach', 'beetroot', 'broccoli', 'cucumber']}
child3: {'likes': ['carrot', 'zucchini'], 'neutral': ['pumpkin', 'red pepper', 'beetroot', 'cabbage'], 'dislikes': ['tomato', 'corn', 'onion', 'cauliflower', 'spinach', 'chard', 'peas', 'lettuce', 'broccoli', 'eggplant', 'cucumber', 'green beans']}
child4: {'likes': ['red pepper', 'broccoli', 'green beans'], 'neutral': ['tomato', 'corn', 'pumpkin', 'zucchini', 'chard'], 'dislikes': ['carrot', 'onion', 'cauliflower', 'spinach', 'peas', 'beetroot', 'lettuce', 'eggplant', 'cucumber', 'cabbage']}
chil

In [24]:
# Feedback collection function
def get_preferences():
    children_data = {
                    'child1': {'likes': ['tomato', 'corn', 'pumpkin', 'carrot', 'peas', 'beetroot', 'cucumber'], 'neutral': ['onion', 'red pepper', 'chard', 'lettuce', 'broccoli'], 'dislikes': ['cauliflower', 'zucchini', 'spinach', 'eggplant', 'cabbage', 'green beans']},
                    'child2': {'likes': ['corn'], 'neutral': ['tomato', 'carrot', 'onion', 'red pepper', 'spinach', 'chard', 'beetroot', 'lettuce', 'broccoli', 'eggplant', 'cucumber', 'cabbage', 'green beans'], 'dislikes': ['pumpkin', 'cauliflower', 'zucchini', 'peas']},
                    'child3': {'likes': ['tomato', 'corn', 'pumpkin', 'carrot', 'eggplant', 'cucumber'], 'neutral': ['onion', 'red pepper', 'zucchini', 'spinach', 'chard', 'beetroot'], 'dislikes': ['cauliflower', 'peas', 'lettuce', 'broccoli', 'cabbage', 'green beans']},
                    'child4': {'likes': ['corn', 'pumpkin', 'carrot'], 'neutral': ['tomato', 'chard', 'peas', 'beetroot', 'lettuce', 'broccoli'], 'dislikes': ['onion', 'red pepper', 'cauliflower', 'zucchini', 'spinach', 'eggplant', 'cucumber', 'cabbage', 'green beans']},
                    'child5': {'likes': ['carrot', 'chard', 'eggplant'], 'neutral': ['tomato', 'corn', 'pumpkin', 'onion', 'zucchini', 'spinach', 'peas', 'broccoli'], 'dislikes': ['red pepper', 'cauliflower', 'beetroot', 'lettuce', 'cucumber', 'cabbage', 'green beans']},
                    'child6': {'likes': ['corn', 'pumpkin', 'onion', 'zucchini', 'peas', 'green beans'], 'neutral': ['tomato', 'red pepper', 'cauliflower', 'beetroot', 'lettuce', 'broccoli'], 'dislikes': ['carrot', 'spinach', 'chard', 'eggplant', 'cucumber', 'cabbage']},
                    'child7': {'likes': ['corn', 'pumpkin', 'carrot'], 'neutral': ['tomato', 'onion', 'red pepper', 'zucchini', 'spinach', 'peas', 'lettuce', 'cucumber', 'cabbage'], 'dislikes': ['cauliflower', 'chard', 'beetroot', 'broccoli', 'eggplant', 'green beans']},
                    'child8': {'likes': [], 'neutral': ['corn', 'pumpkin', 'carrot', 'red pepper', 'zucchini', 'spinach', 'beetroot', 'cabbage'], 'dislikes': ['tomato', 'onion', 'cauliflower', 'chard', 'peas', 'lettuce', 'broccoli', 'eggplant', 'cucumber', 'green beans']},
                    'child9': {'likes': ['tomato', 'corn', 'pumpkin', 'carrot', 'red pepper', 'spinach', 'broccoli'], 'neutral': ['onion', 'zucchini', 'chard', 'peas', 'beetroot', 'cabbage'], 'dislikes': ['cauliflower', 'lettuce', 'eggplant', 'cucumber', 'green beans']},
                    'child10': {'likes': ['tomato', 'corn', 'carrot', 'cauliflower', 'zucchini', 'chard', 'peas'], 'neutral': ['pumpkin', 'red pepper', 'spinach', 'cabbage', 'green beans'], 'dislikes': ['onion', 'beetroot', 'lettuce', 'broccoli', 'eggplant', 'cucumber']},
                    'child11': {'likes': ['tomato', 'corn', 'pumpkin', 'carrot'], 'neutral': ['red pepper', 'cauliflower', 'zucchini', 'chard', 'beetroot', 'broccoli', 'eggplant', 'green beans'], 'dislikes': ['onion', 'spinach', 'peas', 'lettuce', 'cucumber', 'cabbage']},
                    'child12': {'likes': ['tomato', 'corn', 'carrot', 'peas', 'broccoli'], 'neutral': ['pumpkin', 'cauliflower', 'zucchini', 'beetroot', 'lettuce', 'eggplant', 'cucumber'], 'dislikes': ['onion', 'red pepper', 'spinach', 'chard', 'cabbage', 'green beans']},
                    'child13': {'likes': ['tomato', 'carrot', 'red pepper', 'peas'], 'neutral': ['corn', 'pumpkin', 'onion', 'cauliflower', 'spinach', 'eggplant', 'cabbage', 'green beans'], 'dislikes': ['zucchini', 'chard', 'beetroot', 'lettuce', 'broccoli', 'cucumber']},
                    'child14': {'likes': ['tomato', 'corn', 'carrot', 'zucchini', 'lettuce'], 'neutral': ['pumpkin', 'onion', 'chard', 'eggplant', 'cabbage'], 'dislikes': ['red pepper', 'cauliflower', 'spinach', 'peas', 'beetroot', 'broccoli', 'cucumber', 'green beans']},
                    'child15': {'likes': ['corn', 'pumpkin', 'red pepper', 'lettuce', 'broccoli'], 'neutral': ['tomato', 'carrot', 'onion', 'zucchini', 'spinach', 'chard', 'peas', 'cucumber'], 'dislikes': ['cauliflower', 'beetroot', 'eggplant', 'cabbage', 'green beans']},
                    'child16': {'likes': ['pumpkin', 'onion', 'spinach', 'chard', 'peas', 'cucumber'], 'neutral': ['carrot', 'red pepper', 'zucchini', 'beetroot', 'broccoli', 'eggplant'], 'dislikes': ['tomato', 'corn', 'cauliflower', 'lettuce', 'cabbage', 'green beans']},
                    'child17': {'likes': ['tomato', 'pumpkin', 'carrot', 'onion', 'red pepper', 'chard'], 'neutral': ['zucchini', 'spinach', 'peas', 'beetroot', 'eggplant'], 'dislikes': ['corn', 'cauliflower', 'lettuce', 'broccoli', 'cucumber', 'cabbage', 'green beans']},
                    'child18': {'likes': ['tomato', 'corn', 'pumpkin', 'carrot', 'onion', 'zucchini'], 'neutral': ['red pepper', 'peas', 'lettuce', 'broccoli'], 'dislikes': ['cauliflower', 'spinach', 'chard', 'beetroot', 'eggplant', 'cucumber', 'cabbage', 'green beans']},
                    'child19': {'likes': ['tomato', 'corn', 'onion', 'red pepper', 'zucchini', 'peas'], 'neutral': ['pumpkin', 'carrot', 'cauliflower', 'chard', 'broccoli', 'cabbage'], 'dislikes': ['spinach', 'beetroot', 'lettuce', 'eggplant', 'cucumber', 'green beans']},
                    'child20': {'likes': ['tomato', 'pumpkin', 'onion', 'red pepper'], 'neutral': ['corn', 'carrot', 'cauliflower', 'zucchini', 'spinach', 'peas', 'lettuce', 'eggplant', 'cucumber'], 'dislikes': ['chard', 'beetroot', 'broccoli', 'cabbage', 'green beans']},
                    'child21': {'likes': ['tomato', 'onion', 'red pepper', 'chard'], 'neutral': ['corn', 'pumpkin', 'cauliflower', 'zucchini', 'beetroot', 'cucumber', 'cabbage', 'green beans'], 'dislikes': ['carrot', 'spinach', 'peas', 'lettuce', 'broccoli', 'eggplant']},
                    'child22': {'likes': ['tomato', 'corn', 'zucchini', 'broccoli'], 'neutral': ['pumpkin', 'onion', 'spinach', 'lettuce', 'cucumber', 'cabbage'], 'dislikes': ['carrot', 'red pepper', 'cauliflower', 'chard', 'peas', 'beetroot', 'eggplant', 'green beans']},
                    'child23': {'likes': ['carrot', 'chard'], 'neutral': ['tomato', 'corn', 'pumpkin', 'onion', 'red pepper', 'spinach', 'lettuce', 'cucumber', 'cabbage'], 'dislikes': ['cauliflower', 'zucchini', 'peas', 'beetroot', 'broccoli', 'eggplant', 'green beans']},
                    'child24': {'likes': ['tomato', 'corn', 'pumpkin', 'carrot', 'red pepper', 'chard', 'peas', 'beetroot', 'eggplant'], 'neutral': ['onion', 'zucchini', 'spinach', 'broccoli'], 'dislikes': ['cauliflower', 'lettuce', 'cucumber', 'cabbage', 'green beans']},
                    'child25': {'likes': ['carrot'], 'neutral': ['tomato', 'corn', 'pumpkin', 'red pepper', 'cauliflower', 'zucchini', 'spinach', 'peas', 'lettuce', 'broccoli', 'cucumber'], 'dislikes': ['onion', 'chard', 'beetroot', 'eggplant', 'cabbage', 'green beans']},
                    'child26': {'likes': ['tomato', 'corn', 'pumpkin', 'carrot', 'red pepper', 'lettuce', 'cabbage', 'green beans'], 'neutral': ['cauliflower', 'broccoli', 'eggplant'], 'dislikes': ['onion', 'zucchini', 'spinach', 'chard', 'peas', 'beetroot', 'cucumber']},
                    'child27': {'likes': ['pumpkin', 'cucumber'], 'neutral': ['tomato', 'corn', 'carrot', 'cauliflower', 'zucchini', 'chard', 'eggplant'], 'dislikes': ['onion', 'red pepper', 'spinach', 'peas', 'beetroot', 'lettuce', 'broccoli', 'cabbage', 'green beans']},
                    'child28': {'likes': ['tomato', 'pumpkin', 'carrot', 'onion', 'cauliflower', 'spinach', 'peas', 'cabbage'], 'neutral': ['broccoli', 'eggplant'], 'dislikes': ['corn', 'red pepper', 'zucchini', 'chard', 'beetroot', 'lettuce', 'cucumber', 'green beans']},
                    'child29': {'likes': ['carrot'], 'neutral': ['corn', 'pumpkin', 'onion', 'zucchini', 'cucumber'], 'dislikes': ['tomato', 'red pepper', 'cauliflower', 'spinach', 'chard', 'peas', 'beetroot', 'lettuce', 'broccoli', 'eggplant', 'cabbage', 'green beans']},
                    'child30': {'likes': ['tomato', 'corn', 'carrot', 'red pepper', 'broccoli', 'cucumber', 'cabbage'], 'neutral': ['pumpkin', 'onion', 'cauliflower', 'zucchini', 'peas', 'beetroot', 'green beans'], 'dislikes': ['spinach', 'chard', 'lettuce', 'eggplant']}}
    return children_data


### Child Feedback

#### Example Meal Plan
#### [onions, red pepper, egg plant, chicken, cauliflower, potatoes]

Generated using OpenAI GPT, the comments are crafted to diverge from the stored user preferences. These comments play a crucial role as they can update the preference list. Additionally, the comments are designed to discuss multiple ingredients, allowing us to assess if the sentiment towards each ingredient can be accurately measured.

In [25]:
# Feedback collection function
def collect_feedback():
    feedback = {
        "child1": {"satisfied": "neutral", "comments": "Didn't like the cauliflower and red pepper in the dish, but the chicken was tasty."},
        "child2": {"satisfied": "negative", "comments": "Did not enjoy the cauliflower and potatoes."},
        "child3": {"satisfied": "positive", "comments": "Enjoyed the chicken and cauliflower, but was okay with the onion."},
        "child4": {"satisfied": "positive", "comments": "Loved the chicken, but didn't like the red pepper and cauliflower."},
        "child5": {"satisfied": "neutral", "comments": "The cauliflower was great, but the potatoes were just okay."},
        "child6": {"satisfied": "neutral", "comments": "Didn't enjoy the onion, but the chicken and potatoes were okay."},
        "child7": {"satisfied": "positive", "comments": "Loved the chicken and potatoes, but not the cauliflower."},
        "child8": {"satisfied": "neutral", "comments": "Loved the chicken and potatoes, but the cauliflower was not appealing."},
        "child9": {"satisfied": "positive", "comments": "Enjoyed the chicken, but the cauliflower was not liked."},
        "child10": {"satisfied": "negative", "comments": "Didn't like the onion and red pepper stir-fry."},
        "child11": {"satisfied": "positive", "comments": "Really liked the chicken with potatoes and the red pepper was tasty."},
        "child12": {"satisfied": "neutral", "comments": "Didn't like the cauliflower in the dish, but the chicken was fine."},
        "child13": {"satisfied": "positive", "comments": "Enjoyed the chicken and potatoes, but not the cauliflower."},
        "child14": {"satisfied": "negative", "comments": "Didn't like the cauliflower and potatoes."},
        "child15": {"satisfied": "neutral", "comments": "The chicken and potatoes were amazing, but didn't enjoy the cauliflower much."},
        "child16": {"satisfied": "neutral", "comments": "Loved the chicken and potatoes, but not the cauliflower."},
        "child17": {"satisfied": "negative", "comments": "Didn't enjoy the chicken much, but the potatoes were okay."},
        "child18": {"satisfied": "positive", "comments": "The chicken and potatoes dish was great."},
        "child19": {"satisfied": "negative", "comments": "Didn't like the cauliflower."},
        "child20": {"satisfied": "positive", "comments": "Enjoyed the chicken and potatoes."},
        "child21": {"satisfied": "positive", "comments": "Loved the chicken and cauliflower."},
        "child22": {"satisfied": "negative", "comments": "Didn't like the potatoes and the cauliflower."},
        "child23": {"satisfied": "positive", "comments": "Enjoyed the chicken and potatoes, but the cauliflower was okay."},
        "child24": {"satisfied": "negative", "comments": "Didn't like the cauliflower and red pepper in the dish."},
        "child25": {"satisfied": "neutral", "comments": "Didn't like the chicken and cauliflower, but the potatoes were okay."},
        "child26": {"satisfied": "neutral", "comments": "Enjoyed the chicken and cauliflower, but didn't like the red pepper."},
        "child27": {"satisfied": "negative", "comments": "Didn't like the cauliflower."},
        "child28": {"satisfied": "neutral", "comments": "Loved the chicken and potatoes, but the red pepper was not liked."},
        "child29": {"satisfied": "neutral", "comments": "Didn't like the chicken much and the potatoes were okay."},
        "child30": {"satisfied": "positive", "comments": "Enjoyed the chicken and potatoes."},
    }
    return feedback


##  Sentiment analysis

Use to take comments and update preferences, by finding what ingredients they speak positively about and don't 

In [59]:
def analyze_sentiment():
    pass

import re

# Function to update children's data with new preferences based on feedback
def extract_preferences_and_update_data(preferences, feedback):
    # List of ingredients to check in feedback
    ingredients = ['onion', 'red pepper', 'chicken', 'cauliflower', 'potatoes']
    changes = []
    neutral_threshold = 0.1  # Threshold for considering an ingredient neutral
    
    # Iterate over each child's feedback
    for child, fb in feedback.items():
        # Split comments into sentences based on punctuation
        comments = re.split(r'[,.!?]', fb["comments"].lower())
        
        # Analyze each comment's sentiment
        for sentence in comments:
            if sentence.strip():  # Check if the sentence is not empty
                polarity = analyze_sentiment(sentence.strip())
                
                # Check for mentions of each ingredient in the sentence
                for ingredient in ingredients:
                    if ingredient in sentence:
                        change = {"child": child, "ingredient": ingredient, "change": ""}
                        
                        # Determine the appropriate category based on polarity
                        if polarity > neutral_threshold:
                            if ingredient not in preferences[child]["likes"]:
                                preferences[child]["likes"].append(ingredient)
                                change["change"] = "added to likes"
                        elif polarity < -neutral_threshold:
                            if ingredient not in preferences[child]["dislikes"]:
                                preferences[child]["dislikes"].append(ingredient)
                                change["change"] = "added to dislikes"
                        else:
                            if ingredient not in preferences[child]["neutral"]:
                                preferences[child]["neutral"].append(ingredient)
                                change["change"] = "added to neutral"
                        
                        # Remove ingredient from other lists
                        if change["change"]:
                            if change["change"] != "added to likes" and ingredient in preferences[child]["likes"]:
                                preferences[child]["likes"].remove(ingredient)
                                change["change"] += ", removed from likes"
                            if change["change"] != "added to dislikes" and ingredient in preferences[child]["dislikes"]:
                                preferences[child]["dislikes"].remove(ingredient)
                                change["change"] += ", removed from dislikes"
                            if change["change"] != "added to neutral" and ingredient in preferences[child]["neutral"]:
                                preferences[child]["neutral"].remove(ingredient)
                                change["change"] += ", removed from neutral"
                            
                            changes.append(change)
    
    return changes, preferences

# Function to display the changes made to children's preferences
def display_changes(changes):
    for change in changes:
        print(f"Child {change['child']} had {change['ingredient']} {change['change']}.")

### Text Blob Method

Fast but doesn't completley capture all the context around ingredients

In [60]:
from textblob import TextBlob

# Function to analyze the sentiment of a comment
def analyze_sentiment(comment):
    # Create a TextBlob object from the comment
    analysis = TextBlob(comment)
    # Get the polarity of the comment's sentiment (-1 to 1)
    polarity = analysis.sentiment.polarity
    # Return the polarity
    return polarity

# Assume feedback is collected and children_data is retrieved from some source
feedback = collect_feedback()
children_data = get_preferences()

# Update children_data with new preferences and get the list of changes
changes,updated_children_data = extract_preferences_and_update_data(children_data, feedback)

# Display the changes
display_changes(changes)

Child child1 had cauliflower added to neutral, removed from dislikes, removed from neutral.
Child child1 had chicken added to dislikes.
Child child2 had potatoes added to dislikes.
Child child3 had chicken added to neutral.
Child child3 had cauliflower added to neutral, removed from dislikes, removed from neutral.
Child child3 had onion added to likes, removed from neutral.
Child child4 had chicken added to neutral.
Child child4 had red pepper added to neutral, removed from dislikes, removed from neutral.
Child child4 had cauliflower added to neutral, removed from dislikes, removed from neutral.
Child child5 had cauliflower added to likes, removed from dislikes.
Child child5 had potatoes added to likes.
Child child6 had chicken added to neutral.
Child child6 had potatoes added to neutral.
Child child7 had chicken added to neutral.
Child child7 had potatoes added to neutral.
Child child7 had cauliflower added to neutral, removed from dislikes, removed from neutral.
Child child8 had chic

### Hugging Face Transformer Model

In [62]:
from transformers import pipeline
import torch

# Check if GPU is available and set device accordingly
device = 0 if torch.cuda.is_available() else -1

# Load the sentiment analysis pipeline with a specific model
sentiment_analyzer = pipeline('sentiment-analysis', model="distilbert-base-uncased-finetuned-sst-2-english", device=device)

# Function to analyze the sentiment of each sentence using BERT
def analyze_sentiment(sentence):
    result = sentiment_analyzer(sentence)
    label = result[0]['label']
    score = result[0]['score']
    
    # Determine the polarity based on the sentiment label
    if label == 'POSITIVE':
        polarity = score
    else:
        polarity = -score
    return polarity

# Collect feedback and get initial preferences (these functions are assumed to be defined elsewhere)
feedback = collect_feedback()
preferences = get_preferences()

# Update children_data and get the list of changes
changes, updated_preferences = extract_preferences_and_update_data(preferences, feedback)

# # Display the changes
display_changes(changes)

Child child1 had red pepper added to dislikes, removed from neutral.
Child child1 had chicken added to likes.
Child child2 had potatoes added to dislikes.
Child child3 had chicken added to likes.
Child child3 had cauliflower added to likes, removed from dislikes.
Child child3 had onion added to likes, removed from neutral.
Child child4 had chicken added to likes.
Child child5 had cauliflower added to likes, removed from dislikes.
Child child5 had potatoes added to likes.
Child child6 had onion added to dislikes, removed from likes, removed from dislikes.
Child child6 had chicken added to likes.
Child child6 had potatoes added to likes.
Child child7 had chicken added to likes.
Child child7 had potatoes added to likes.
Child child8 had chicken added to likes.
Child child8 had potatoes added to likes.
Child child9 had chicken added to likes.
Child child10 had red pepper added to dislikes, removed from neutral.
Child child11 had red pepper added to likes, removed from neutral.
Child child1

## Collaborative Filtering

In [65]:
def get_ingredient_data():
    # Example expanded ingredient data with ingredient features
    return {
        "tomato": {"type": "vegetable", "color": "red", "taste": "savory"},
        "corn": {"type": "vegetable", "color": "yellow", "taste": "sweet"},
        "pumpkin": {"type": "vegetable", "color": "orange", "taste": "sweet"},
        "carrot": {"type": "vegetable", "color": "orange", "taste": "sweet"},
        "peas": {"type": "vegetable", "color": "green", "taste": "sweet"},
        "beetroot": {"type": "vegetable", "color": "purple", "taste": "earthy"},
        "cucumber": {"type": "vegetable", "color": "green", "taste": "mild"},
        "chicken": {"type": "protein", "color": "white", "taste": "savory"},
        "onion": {"type": "vegetable", "color": "white", "taste": "pungent"},
        "chard": {"type": "vegetable", "color": "green", "taste": "bitter"},
        "lettuce": {"type": "vegetable", "color": "green", "taste": "mild"},
        "broccoli": {"type": "vegetable", "color": "green", "taste": "bitter"},
        "cauliflower": {"type": "vegetable", "color": "white", "taste": "mild"},
        "zucchini": {"type": "vegetable", "color": "green", "taste": "mild"},
        "spinach": {"type": "vegetable", "color": "green", "taste": "bitter"},
        "eggplant": {"type": "vegetable", "color": "purple", "taste": "mild"},
        "cabbage": {"type": "vegetable", "color": "green", "taste": "mild"},
        "green beans": {"type": "vegetable", "color": "green", "taste": "mild"},
        "red pepper": {"type": "vegetable", "color": "red", "taste": "sweet"},
        "potatoes": {"type": "vegetable", "color": "brown", "taste": "starchy"},
        "pasta": {"type": "grain", "color": "yellow", "taste": "savory"},
        "orange": {"type": "fruit", "color": "orange", "taste": "sweet"},
        "turkey": {"type": "protein", "color": "white", "taste": "savory"},
        "kiwi": {"type": "fruit", "color": "green", "taste": "sweet"},
        "apple": {"type": "fruit", "color": "red", "taste": "sweet"},
        "banana": {"type": "fruit", "color": "yellow", "taste": "sweet"},
        "grapes": {"type": "fruit", "color": "purple", "taste": "sweet"},
        "strawberry": {"type": "fruit", "color": "red", "taste": "sweet"},
        "blueberry": {"type": "fruit", "color": "blue", "taste": "sweet"},
        "mango": {"type": "fruit", "color": "orange", "taste": "sweet"},
        "pineapple": {"type": "fruit", "color": "yellow", "taste": "sweet"},
        "watermelon": {"type": "fruit", "color": "green", "taste": "sweet"},
        "peach": {"type": "fruit", "color": "orange", "taste": "sweet"},
        "pear": {"type": "fruit", "color": "green", "taste": "sweet"},
        "plum": {"type": "fruit", "color": "purple", "taste": "sweet"},
        "raspberry": {"type": "fruit", "color": "red", "taste": "sweet"},
        "blackberry": {"type": "fruit", "color": "black", "taste": "sweet"},
        "apricot": {"type": "fruit", "color": "orange", "taste": "sweet"},
        "avocado": {"type": "fruit", "color": "green", "taste": "creamy"},
        "cherry": {"type": "fruit", "color": "red", "taste": "sweet"},
        "fig": {"type": "fruit", "color": "purple", "taste": "sweet"},
        "lime": {"type": "fruit", "color": "green", "taste": "sour"},
        "lemon": {"type": "fruit", "color": "yellow", "taste": "sour"},
        "tangerine": {"type": "fruit", "color": "orange", "taste": "sweet"},
        "pomegranate": {"type": "fruit", "color": "red", "taste": "sweet"},
        "grapefruit": {"type": "fruit", "color": "pink", "taste": "bitter"},
        "bell pepper": {"type": "vegetable", "color": "green", "taste": "mild"},
        "garlic": {"type": "vegetable", "color": "white", "taste": "pungent"},
        "ginger": {"type": "vegetable", "color": "brown", "taste": "spicy"},
        "kale": {"type": "vegetable", "color": "green", "taste": "bitter"},
        "mushroom": {"type": "fungi", "color": "white", "taste": "umami"},
        "celery": {"type": "vegetable", "color": "green", "taste": "mild"},
        "sweet potato": {"type": "vegetable", "color": "orange", "taste": "sweet"},
        "radish": {"type": "vegetable", "color": "red", "taste": "peppery"},
        "turnip": {"type": "vegetable", "color": "white", "taste": "peppery"},
        "parsnip": {"type": "vegetable", "color": "white", "taste": "sweet"},
        "butternut squash": {"type": "vegetable", "color": "orange", "taste": "sweet"},
        "quinoa": {"type": "grain", "color": "white", "taste": "nutty"},
        "barley": {"type": "grain", "color": "brown", "taste": "nutty"},
        "rice": {"type": "grain", "color": "white", "taste": "mild"},
        "lentils": {"type": "legume", "color": "brown", "taste": "earthy"},
        "beans": {"type": "legume", "color": "varied", "taste": "earthy"},
        "chickpeas": {"type": "legume", "color": "beige", "taste": "nutty"},
        "soybeans": {"type": "legume", "color": "green", "taste": "earthy"},
        "tofu": {"type": "protein", "color": "white", "taste": "bland"},
        "salmon": {"type": "protein", "color": "pink", "taste": "savory"},
        "beef": {"type": "protein", "color": "red", "taste": "savory"},
        "pork": {"type": "protein", "color": "white", "taste": "savory"},
        "lamb": {"type": "protein", "color": "red", "taste": "savory"},
        "shrimp": {"type": "protein", "color": "pink", "taste": "savory"},
        "crab": {"type": "protein", "color": "white", "taste": "savory"},
        "lobster": {"type": "protein", "color": "red", "taste": "savory"},
        "oyster": {"type": "protein", "color": "gray", "taste": "savory"},
        "scallop": {"type": "protein", "color": "white", "taste": "savory"},
        "egg": {"type": "protein", "color": "white", "taste": "savory"},
        "milk": {"type": "dairy", "color": "white", "taste": "creamy"},
        "cheese": {"type": "dairy", "color": "varied", "taste": "savory"},
        "yogurt": {"type": "dairy", "color": "white", "taste": "tangy"}
    }

In [99]:
import numpy as np

def get_utility_matrix(children, ingredients, preferences):
    
    # Initialize utility matrix with zeros
    utility_matrix = np.zeros((len(children), len(ingredients)))
    
    # Fill the utility matrix based on preferences
    for i, child in enumerate(children):
        for j, ingredient in enumerate(ingredients):
            if ingredient in preferences[child]["likes"]:
                utility_matrix[i, j] = 5
            elif ingredient in preferences[child]["neutral"]:
                utility_matrix[i, j] = 3
            elif ingredient in preferences[child]["dislikes"]:
                utility_matrix[i, j] = 1
    return utility_matrix

In [126]:
# Predict preferences for a target child using user-based collaborative filtering
def predict_preferences_user_based(target_child_index, user_similarity_matrix, utility_matrix, ingredients):
    # Calculate the weighted sum of preferences from similar users
    weighted_sum = np.dot(user_similarity_matrix[target_child_index], utility_matrix)
    
    # Sum of weights for normalization
    sum_of_weights = np.sum(user_similarity_matrix[target_child_index])
    
    # Debugging output to inspect values
    print(f"Weighted Sum for child {target_child_index}: {weighted_sum}")
    print(f"Sum of Weights for child {target_child_index}: {sum_of_weights}")
    
    # Normalize the weighted sum to get predicted preferences
    predicted_preferences = weighted_sum / sum_of_weights if sum_of_weights != 0 else np.zeros(len(ingredients))
    
    return predicted_preferences

In [127]:
# Predict preferences for a target child using item-based collaborative filtering
def predict_preferences_item_based(target_child_index, item_similarity_matrix, utility_matrix):
    # Retrieve the preferences of the target child
    target_child_preferences = utility_matrix[target_child_index]
    
    # Calculate the weighted sum of item similarities
    # Here we multiply the item similarity matrix with the child's preference vector
    weighted_sum = np.dot(item_similarity_matrix, target_child_preferences)
    
    # Sum of weights for normalization
    sum_of_weights = np.sum(item_similarity_matrix, axis=1)
    
    print(f"Weighted Sum for child {target_child_index}: {weighted_sum}")
    print(f"Sum of Weights for child {target_child_index}: {sum_of_weights}")
    
    # Avoid division by zero by using numpy's divide function with the 'where' parameter
    predicted_preferences = np.divide(weighted_sum, sum_of_weights, out=np.zeros_like(weighted_sum), where=sum_of_weights != 0)
    
    return predicted_preferences

Each entry in the item similarity matrix represents the cosine similarity between two ingredients. The value ranges from -1 to 1, where 1 indicates maximum similarity, 0 indicates no similarity, and -1 indicates maximum dissimilarity.

Higher similarity values between ingredients suggest that children who like (or dislike) one ingredient tend to have similar preferences for the other ingredient.

The predicted preferences array represents the estimated ratings a child would give to each ingredient, based on their current preferences and the similarities between ingredients.

Higher predicted preference values indicate a higher likelihood that the child will like the ingredient.

### User Based Collaborative Filtering

In [128]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
    
children = list(get_child_database().keys())
ingredients = list(get_ingredient_data().keys())

# Get the utility matrix based on updated preferences
utility_matrix = get_utility_matrix(children, ingredients, updated_preferences)

# Calculate similarity matrix
user_similarity_matrix = cosine_similarity(utility_matrix)

# print("User Similarity Matrix:")
# print(user_similarity_matrix)

# Example usage:
target_child_index = 0  # You can change this index to test with different children
predicted_preferences = predict_preferences_user_based(target_child_index, user_similarity_matrix, utility_matrix, ingredients)
print(f"Predicted preferences for child {children[target_child_index]}: {predicted_preferences}")

Weighted Sum for child 0: [96.13137342 96.91129795 96.54457608 99.77379545 69.37869492 50.81924013
 57.09490266 79.94205021 69.13913863 61.92764696 51.53332147 61.51405776
 51.49732477 69.40267743 55.12955421 50.75121992 50.20371596 39.39070917
 64.94107934 61.01654703  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
Sum of Weights for child 0: 23.940777116748748
Predicted preferences for child child1: [4.015382

### Item Based Collaborative Filtering

In [130]:
# List of children and ingredients
children = list(get_child_database().keys())
ingredients = list(get_ingredient_data().keys())

# Get the utility matrix based on updated preferences
utility_matrix = get_utility_matrix(children, ingredients, updated_preferences)

# Calculate item similarity matrix (transpose the utility matrix for item-based similarity)
item_similarity_matrix = cosine_similarity(utility_matrix.T)

print("Item Similarity Matrix:")
print(item_similarity_matrix)


# Example usage: Predict preferences for the first child
target_child_index = 0
predicted_preferences = predict_preferences_item_based(target_child_index, item_similarity_matrix, utility_matrix)
print(f"Predicted preferences for {children[target_child_index]}:")
print(predicted_preferences)

Item Similarity Matrix:
[[1.         0.93258427 0.92406125 ... 0.         0.         0.        ]
 [0.93258427 1.         0.90885037 ... 0.         0.         0.        ]
 [0.92406125 0.90885037 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
Weighted Sum for child 0: [50.82489349 50.34318924 51.45303278 50.2898715  47.34373111 47.34245188
 45.37808906 45.59504171 47.02070294 45.17462447 44.67686438 47.3149449
 42.10683429 46.99902773 45.97092616 44.81600626 44.18815913 42.84333196
 44.81464053 38.36697836  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.

#### Both the item based and user based predicted preferences are sparse as in total only 20 ingredients have been rated so far. An effort to add more features will not be done to see the result.

### User Based Collaborative Filtering with Added Features

User-based collaborative filtering predicts a user's preferences based on the preferences of similar users.
It assumes that if user A has a similar taste to user B, then user A is likely to prefer items that user B likes.

In [133]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Get child data
child_data = get_child_database()

# Extract lists of children and ingredients
children = list(child_data.keys())
ingredients = list(get_ingredient_data().keys())

# Get the utility matrix based on updated preferences
utility_matrix = get_utility_matrix(children, ingredients, updated_preferences)

# Prepare data for encoding
# Extract relevant features from the child data
ages = [child_data[child]['age'] for child in children]
genders = [child_data[child]['gender'] for child in children]
health_considerations = [child_data[child]['health_consideration'] for child in children]
favorite_cuisines = [child_data[child]['favorite_cuisine'] for child in children]

# Combine all features into a single array
# .T transposes the array to match the expected input shape for encoding
combined_features = np.array([ages, genders, health_considerations, favorite_cuisines]).T

# One-hot encode the categorical features (genders, health_considerations, favorite_cuisines)
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(combined_features[:, 1:]).toarray()  # Skip 'ages' for now

# Scale the age feature to normalize it
scaler = StandardScaler()
scaled_ages = scaler.fit_transform(np.array(ages).reshape(-1, 1))

# Combine scaled ages with encoded categorical features
user_features = np.hstack((scaled_ages, encoded_features))

# Calculate user similarity matrix using combined user features and utility matrix
# Concatenate user features with the utility matrix for similarity calculation
user_similarity_matrix = cosine_similarity(np.hstack((user_features, utility_matrix)))

# Uncomment the following lines to inspect the user similarity matrix
# print("User Similarity Matrix:")
# print(user_similarity_matrix)


In [136]:
# Example usage: Predict preferences for the first child
target_child_index = 0
predicted_preferences = predict_preferences_user_based(target_child_index, user_similarity_matrix, utility_matrix, ingredients)
print(f"Predicted preferences for {children[target_child_index]}:")
print(predicted_preferences)

Weighted Sum for child 0: [95.38832534 96.13291625 95.7778572  99.00068954 68.9364016  50.45817366
 56.64105325 79.34418465 68.59977149 61.46408178 51.09491544 61.02164055
 51.11327707 68.8546952  54.69881114 50.36776046 49.76487169 39.07261374
 64.41007943 60.50473799  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
Sum of Weights for child 0: 23.741567003397844
Predicted preferences for child1:
[4.01777715 4.0

### Item Based Collaborative Filtering with added features

In [137]:
# Import necessary libraries
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder

# List of children and ingredients
# Assuming `feedback` is a dictionary with children as keys and their feedback/preferences as values
children = list(feedback.keys())
ingredients = list(get_ingredient_data().keys())

# Get the utility matrix based on updated preferences
utility_matrix = get_utility_matrix(children, ingredients, updated_preferences)

# Get ingredient data
ingredient_data = get_ingredient_data()

# Prepare data for one-hot encoding
# Extract relevant features from the ingredient data
types = [ingredient_data[ingredient]['type'] for ingredient in ingredients]
colors = [ingredient_data[ingredient]['color'] for ingredient in ingredients]
tastes = [ingredient_data[ingredient]['taste'] for ingredient in ingredients]

# Combine all features into a single array
# .T transposes the array to match the expected input shape for encoding
combined_features = np.array([types, colors, tastes]).T

# One-hot encode the features (types, colors, tastes)
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(combined_features).toarray()

# Calculate item similarity matrix using encoded features
item_similarity_matrix = cosine_similarity(encoded_features)

# Uncomment the following lines to inspect the item similarity matrix
# print("Item Similarity Matrix:")
# print(item_similarity_matrix)

# Predict preferences for a target child using item-based collaborative filtering
def predict_preferences_item_based(target_child_index):
    # Retrieve the preferences of the target child from the utility matrix
    target_child_preferences = utility_matrix[target_child_index]
    
    # Calculate the weighted sum of item similarities
    weighted_sum = np.dot(target_child_preferences, item_similarity_matrix)
    
    # Sum of weights for normalization
    sum_of_weights = np.sum(item_similarity_matrix, axis=1)
    
    # Normalize the weighted sum to get predicted preferences
    # Use numpy's divide function to handle division by zero
    predicted_preferences = np.divide(weighted_sum, sum_of_weights, out=np.zeros_like(weighted_sum), where=sum_of_weights != 0)
    
    return predicted_preferences

# Example usage: Predict preferences for the first child
target_child_index = 0
predicted_preferences = predict_preferences_item_based(target_child_index)
print(f"Predicted preferences for {children[target_child_index]}:")
print(predicted_preferences)


Predicted preferences for child1:
[1.27272727 1.29032258 1.28787879 1.28787879 1.30666667 1.71052632
 1.57894737 0.53333333 1.375      1.61538462 1.57894737 1.61538462
 1.35714286 1.57894737 1.61538462 1.65909091 1.57894737 1.57894737
 1.19117647 1.58823529 0.625      0.50819672 0.53333333 0.62857143
 0.42857143 0.45614035 0.47368421 0.42857143 0.39622642 0.50819672
 0.45614035 0.62857143 0.50819672 0.62857143 0.47368421 0.42857143
 0.39622642 0.50819672 0.52272727 0.42857143 0.47368421 0.52272727
 0.16129032 0.50819672 0.42857143 0.21875    1.57894737 1.375
 1.58823529 1.61538462 0.47368421 1.57894737 1.28787879 1.42857143
 1.3125     1.13513514 1.28787879 0.375      0.         0.70967742
 0.41666667 0.5        0.         1.07692308 0.4516129  0.48387097
 0.53846154 0.53333333 0.53846154 0.48387097 0.53333333 0.53846154
 0.51724138 0.53333333 0.53333333 0.40909091 0.5        0.42857143]


Item-based collaborative filtering with added features tends to be less sparse and more effective than user-based methods with added features because item features are stable, consistent, and richly described. These characteristics lead to a densely populated item similarity matrix, which can aggregate preferences more effectively and provide better coverage for recommendations. Additionally, item-based methods are more scalable and can handle sparse user interactions more gracefully, making them a preferred choice in many recommendation systems.

### ML Method